PROCESO ML

#### Bibliotecas

In [1]:
import bootcampviztools as bt
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

# WEB SCRAPING SUPERHEROES AND VILLAINS

In [2]:
import lxml
import requests
import pandas as pd
from bs4 import BeautifulSoup

ALFRED

In [35]:
url = 'https://www.superherodb.com/alfred/10-14275/'
response = requests.get(url)
response 

<Response [200]>

In [7]:
# from fake_useragent import UserAgent                #ya que me niega el acceso, hay que ejecutar este código
# ua = UserAgent()
# headers = {'User-Agent': ua.random}
# response = requests.get(url, headers = headers)
# response

<Response [200]>

In [36]:
# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'column col-10 col-md-9 col-sm-12'
    div_column = soup.find('div', class_='column col-10 col-md-9 col-sm-12')
    
    # Extraer el nombre completo del superhéroe (h2 dentro del div encontrado)
    nombre_completo = div_column.find('h2').text.strip()
    
    # Extraer el nombre del superhéroe (primer h1 dentro del div encontrado)
    nombre = div_column.find('h1').text.strip()
    
    # Extraer el universo del superhéroe (primer h3 dentro del div encontrado)
    universo = div_column.find('h3').text.strip()
    
    # Imprimir los resultados
    print(f"Nombre completo: {nombre_completo}")
    print(f"Nombre: {nombre}")
    print(f"Universo: {universo}")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


Nombre completo: Alfred Pennyworth
Nombre: Alfred
Universo: DC Extended Universe


In [25]:
# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'stat-holder stat-rows'
    stat_holder = soup.find('div', class_='stat-holder stat-rows')
    
    if stat_holder:
        # Encontrar todos los elementos div con clase 'stat-bar' dentro de 'stat-holder'
        stat_bars = stat_holder.find_all('div', class_='stat-bar')
        
        # Iterar sobre cada stat-bar para extraer los stats y stat-values
        for stat_bar in stat_bars:
            # Encontrar el stat-value dentro de cada stat-bar
            stat_value = stat_bar.find('div', class_='stat-value')
            if stat_value:
                stat_name = stat_bar.find('label').text.strip()
                stat_value_text = stat_value.text.strip()
                print(f"{stat_name}: {stat_value_text}")
            else:
                print("No se encontró el stat-value dentro de stat-bar")
    else:
        print("No se encontró el elemento stat-holder stat-rows en la página.")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


Intelligence: 0
Strength: 0
Speed: 0
Durability: 0
Power: 0
Combat: 0
Tier: 0
Tier2: 0


AMANDA WALLER 

In [34]:
url = 'https://www.superherodb.com/amanda-waller/10-33322/'
response = requests.get(url)
response 

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'column col-10 col-md-9 col-sm-12'
    div_column = soup.find('div', class_='column col-10 col-md-9 col-sm-12')
    
    # Extraer el nombre completo del superhéroe (h2 dentro del div encontrado)
    nombre_completo = div_column.find('h2').text.strip()
    
    # Extraer el nombre del superhéroe (primer h1 dentro del div encontrado)
    nombre = div_column.find('h1').text.strip()
    
    # Extraer el universo del superhéroe (primer h3 dentro del div encontrado)
    universo = div_column.find('h3').text.strip()
    
    # Imprimir los resultados
    print(f"Nombre completo: {nombre_completo}")
    print(f"Nombre: {nombre}")
    print(f"Universo: {universo}")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'stat-holder stat-rows'
    stat_holder = soup.find('div', class_='stat-holder stat-rows')
    
    if stat_holder:
        # Encontrar todos los elementos div con clase 'stat-bar' dentro de 'stat-holder'
        stat_bars = stat_holder.find_all('div', class_='stat-bar')
        
        # Iterar sobre cada stat-bar para extraer los stats y stat-values
        for stat_bar in stat_bars:
            # Encontrar el stat-value dentro de cada stat-bar
            stat_value = stat_bar.find('div', class_='stat-value')
            if stat_value:
                stat_name = stat_bar.find('label').text.strip()
                stat_value_text = stat_value.text.strip()
                print(f"{stat_name}: {stat_value_text}")
            else:
                print("No se encontró el stat-value dentro de stat-bar")
    else:
        print("No se encontró el elemento stat-holder stat-rows en la página.")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


Nombre completo: Amanda Blake Waller
Nombre: Amanda Waller
Universo: DC Extended Universe
Intelligence: 0
Strength: 0
Speed: 0
Durability: 0
Power: 0
Combat: 0
Tier: 0
Tier2: 0


# DC EXTENDED UNIVERSE 

In [35]:
# Función para extraer la información del superhéroe de la URL dada
def obtener_informacion_heroe(url):
    try:
        # Realizar la solicitud HTTP
        response = requests.get(url)
        
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Parsear el contenido HTML con BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Encontrar el div principal que contiene la información
            div_column = soup.find('div', class_='column col-10 col-md-9 col-sm-12')
            
            # Extraer el nombre completo del superhéroe
            nombre_completo = div_column.find('h2').text.strip() if div_column.find('h2') else "No disponible"
            
            # Extraer el nombre del superhéroe
            nombre = div_column.find('h1').text.strip() if div_column.find('h1') else "No disponible"
            
            # Extraer el universo del superhéroe
            universo = div_column.find('h3').text.strip() if div_column.find('h3') else "No disponible"
            
            # Inicializar un diccionario para las estadísticas
            stats = {
                'nombre_completo': nombre_completo,
                'nombre': nombre,
                'universo': universo,
                'url': url
            }

            # Encontrar el elemento div con clase 'stat-holder stat-rows'
            stat_holder = soup.find('div', class_='stat-holder stat-rows')
            
            if stat_holder:
                # Encontrar todos los elementos div con clase 'stat-bar' dentro de 'stat-holder'
                stat_bars = stat_holder.find_all('div', class_='stat-bar')
                
                # Iterar sobre cada stat-bar para extraer los stats y stat-values
                for stat_bar in stat_bars:
                    # Encontrar el stat-value dentro de cada stat-bar
                    stat_value = stat_bar.find('div', class_='stat-value')
                    if stat_value:
                        stat_name = stat_bar.find('label').text.strip()
                        stat_value_text = stat_value.text.strip()
                        stats[stat_name] = stat_value_text
                    else:
                        print("No se encontró el stat-value dentro de stat-bar")
            else:
                print("No se encontró el elemento stat-holder stat-rows en la página.")
            
            return stats
        else:
            print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error al procesar la URL {url}: {e}")
        return None


In [25]:
# Diccionario 
dc_extended_universe = {
    'Alfred': 'https://www.superherodb.com/alfred/10-14275/#shdb',
    'Amanda Waller': 'https://www.superherodb.com/amanda-waller/10-33322/',
    'Anthea': 'https://www.superherodb.com/anthea/10-54768/#user',
    'Aquaman': 'https://www.superherodb.com/aquaman/10-12533/',
    'Ares': 'https://www.superherodb.com/ares/10-12597/',
    'Atom Smasher': 'https://www.superherodb.com/atom-smasher/10-27865/',
    'Batman': 'https://www.superherodb.com/batman/10-12530/',
    'Black Adam': 'https://www.superherodb.com/black-adam/10-21003/',
    'Black Canary': 'https://www.superherodb.com/black-canary/10-18617/',
    'Black Manta': 'https://www.superherodb.com/black-manta/10-15579/',
    'Black Mask': 'https://www.superherodb.com/black-mask/10-18619/',
    'Bloodsport': 'https://www.superherodb.com/bloodsport/10-21321/',
    'Captain Boomerang': 'https://www.superherodb.com/captain-boomerang/10-15413/',
    'Cassandra Cain': 'https://www.superherodb.com/cassandra-cain/10-18637/',
    'Cheetah': 'https://www.superherodb.com/cheetah/10-20281/',
    'Cyborg': 'https://www.superherodb.com/cyborg/10-12532/',
    'Cyclone': 'https://www.superherodb.com/cyclone/10-27871/',
    'Dark Flash': 'https://www.superherodb.com/dark-flash/10-58454/',
    'Darkseid': 'https://www.superherodb.com/darkseid/10-21002/',
    'Darla Dudley': 'https://www.superherodb.com/darla-dudley/10-19150/#user',
    'Deadshot': 'https://www.superherodb.com/deadshot/10-12599/',
    'Deathstroke': 'https://www.superherodb.com/deathstroke/10-12590/#user',
    'Doctor Fate': 'https://www.superherodb.com/doctor-fate/10-27717/',
    'Doctor Sivana': 'https://www.superherodb.com/doctor-sivana/10-17484/', 
    'Doomsday': 'https://www.superherodb.com/doomsday/10-12587/',
    'Dr. Maru': 'https://www.superherodb.com/dr-maru/10-40291/#user',
    'El Diablo': 'https://www.superherodb.com/el-diablo/10-13967/',
    'Emilia Harcourt': 'https://www.superherodb.com/emilia-harcourt/10-44245/',
    'Enchantress': 'https://www.superherodb.com/enchantress/10-13885/',
    'Envy': 'https://www.superherodb.com/envy/10-27552/',
    'Faora': 'https://www.superherodb.com/faora/10-15086/',
    'Freddy Freeman': 'https://www.superherodb.com/freddy-freeman/10-16682/#user', 
    'General Zod': 'https://www.superherodb.com/general-zod/10-12609/',
    'Gluttony': 'https://www.superherodb.com/gluttony/10-27553/',
    'Green Lantern': 'https://www.superherodb.com/green-lantern/10-27159/',
    'Harley Quinn': 'https://www.superherodb.com/harley-quinn/10-12608/',
    'Hawkman': 'https://www.superherodb.com/hawkman/10-27879/',
    'Hespera': 'https://www.superherodb.com/hespera/10-27883/#user',
    'Huntress': 'https://www.superherodb.com/huntress/10-18621/',
    'Incubus': 'https://www.superherodb.com/incubus/10-15960/',
    'Joker': 'https://www.superherodb.com/joker/10-12707/',
    'Jor-El': 'https://www.superherodb.com/jor-el/10-29663/',
    'Judomaster': 'https://www.superherodb.com/judomaster/10-44243/',
    'Kalypso': 'https://www.superherodb.com/kalypso/10-32522/#user',
    'Karathen': 'https://www.superherodb.com/karathen/10-16145/',
    'Katana': 'https://www.superherodb.com/katana/10-13966/',
    'Killer Croc': 'https://www.superherodb.com/killer-croc/10-13849/',
    'King Nereus': 'https://www.superherodb.com/king-nereus/10-19144/',
    'King Shark': 'https://www.superherodb.com/king-shark/10-27933/',
    'Ladon': 'https://www.superherodb.com/ladon/10-55928/#user', 

}

In [36]:
# Lista para almacenar la información extraída de cada héroe
heroes_info = []

# Iterar sobre cada superhéroe en el diccionario
for heroe, url in dc_extended_universe.items():
    print(f"Obteniendo información para {heroe} con la URL: {url}")
    info = obtener_informacion_heroe(url)
    if info:
        # Agregar el nombre del superhéroe a la información recopilada
        info['heroe'] = heroe
        # Agregar la información a la lista
        heroes_info.append(info)
    print("-" * 40)

# Crear un DataFrame de pandas con la información recopilada
df = pd.DataFrame(heroes_info)


Obteniendo información para Alfred con la URL: https://www.superherodb.com/alfred/10-14275/#shdb
----------------------------------------
Obteniendo información para Amanda Waller con la URL: https://www.superherodb.com/alfred/10-14275/#shdb
----------------------------------------
Obteniendo información para Anthea con la URL: https://www.superherodb.com/anthea/10-54768/#user
----------------------------------------
Obteniendo información para Aquaman con la URL: https://www.superherodb.com/aquaman/10-12533/
----------------------------------------
Obteniendo información para Ares con la URL: https://www.superherodb.com/ares/10-12597/
----------------------------------------
Obteniendo información para Atom Smasher con la URL: https://www.superherodb.com/atom-smasher/10-27865/
----------------------------------------
Obteniendo información para Batman con la URL: https://www.superherodb.com/batman/10-12530/
----------------------------------------
Obteniendo información para Black Ada

In [34]:
df.head(5)

,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,heroe
0,Alfred Pennyworth,Alfred,DC Extended Universe,https://www.superherodb.com/alfred/10-14275/#shdb,0,0,0,0,0,0,0,0,Alfred
1,Alfred Pennyworth,Alfred,DC Extended Universe,https://www.superherodb.com/alfred/10-14275/#shdb,0,0,0,0,0,0,0,0,Amanda Waller
2,,Anthea,DC Extended Universe,https://www.superherodb.com/anthea/10-54768/#user,0,0,0,0,0,0,0,0,Anthea
3,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,0,0,0,0,0,0,0,0,Aquaman
4,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,0,0,0,0,0,0,0,0,Ares
